In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

In [2]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyctaxi-1')

<Experiment: artifact_location=('/home/lu/Learning/mlops_zoomcamp/mlops_zoomcamp/2. Experiment '
 'tracking/mlruns/1'), creation_time=1685414270734, experiment_id='1', last_update_time=1685414270734, lifecycle_stage='active', name='nyctaxi-1', tags={}>

In [4]:
df = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe('./data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

In [7]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [8]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [9]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [10]:
with mlflow.start_run():

    mlflow.set_tag('developer', 'lu')
    mlflow.log_param('train_data_path', './data/yellow_tripdata_2022-01.parquet')
    mlflow.log_param('val_data_path', './data/yellow_tripdata_2022-02.parquet')

    alpha = 0.01
    mlflow.log_param('alpha', alpha)
    lasso = Lasso(alpha)
    lasso.fit(X_train, y_train)

    y_pred = lasso.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

In [11]:
with mlflow.start_run():

    mlflow.set_tag('developer', 'lu')
    mlflow.log_param('train_data_path', './data/yellow_tripdata_2022-01.parquet')
    mlflow.log_param('val_data_path', './data/yellow_tripdata_2022-02.parquet')

    ridge = Ridge()
    ridge.fit(X_train, y_train)

    y_pred = ridge.predict(X_val)

    rsme = mean_squared_error(y_val, y_pred, squared=False)

    mlflow.log_metric('rmse', rmse)

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')

plt.legend()